In [1]:
from pathlib import Path
from g99 import FitsHandler as fh
import numpy as np
from astropy.io import fits

In [16]:


# Define where the directory is located
datadir2 = "/Users/stefa/OneDrive/Documenten/Blaauw images/Spectroscope/Live"
datadir = "C:/Users/stefa/OneDrive/Documenten/Blaauw images/Spectroscope"
procdir = "/Users/stefa/OneDrive/Documenten/Blaauw images/Spectroscope/Process/i"

# Use pathlib.Path to create a pathobject
pathobject = Path(datadir)
pathobject2 = Path(datadir2)

myfitsfiles = []
myfitsfiles2 = []

# Loop over all files in the directory and grab the fits files
for f in pathobject.iterdir():
    if f.suffix.lower() in ['.fits', '.fit', '.fts']:
        myfitsfiles.append(f)

for f in pathobject2.iterdir():
    if f.suffix.lower() in ['.fits', '.fit', '.fts']:
        myfitsfiles2.append(f)


In [ ]:
fits_directory = r"TODO: FITS FILES HERE"
handler = fh(fits_directory)
observation = handler.get_hdus()
bias_files = observation['Bias Frame']

In [ ]:
# Create the image class

class image:
    def __init__(self, filepath, fits_imagetyp='', fits_exptime=0, fits_filter='', fits_date_obs=0, fits_naxis1=0, fits_naxis2=0, pixelsize=18):
        self.filepath = filepath
        self.filename = filepath.name
        self.imagetyp = fits_imagetyp
        self.exptime = fits_exptime
        self.filter = fits_filter
        self.date_obs = fits_date_obs
        self.naxis1 = fits_naxis1
        self.naxis2 = fits_naxis2
        self.pixelsize = pixelsize

myimages = []
myimages2 = []

# Loop over all of the filepaths in myfitsfiles, open each image, give it the image class and append it to the list of images
for filepath in myfitsfiles:
    hdulist = fits.open(filepath)
    hdr = hdulist[0].header
    # Assumes same pixelsize in x and y
    newimage = image(filepath, hdr.get('IMAGETYP'), hdr.get('EXPTIME'), hdr.get('FILTER'), hdr.get('DATE-OBS'), hdr.get('NAXIS1'), hdr.get('NAXIS2'), hdr.get('XPIXSZ'))
    myimages.append(newimage)
    hdulist.close()

for filepath in myfitsfiles2:
    hdulist = fits.open(filepath)
    hdr = hdulist[0].header
    # Assumes same pixelsize in x and y
    newimage = image(filepath, hdr.get('IMAGETYP'), hdr.get('EXPTIME'), hdr.get('FILTER'), hdr.get('DATE-OBS'), hdr.get('NAXIS1'), hdr.get('NAXIS2'), hdr.get('XPIXSZ'))
    myimages2.append(newimage)
    hdulist.close()

    # Comment for self: Dark Frame, Bias Frame, Flat Field, Light Frame

In [ ]:
# Create the masterbias

from matplotlib.pyplot import figure, show, close
from astropy.visualization import ImageNormalize, SquaredStretch

# Loop over all the images in our list and check if they are Bias frames, if they are we append them to the Bias_list
Bias_list = []
count = 0
for im in myimages:
    if 'bias frame' in im.imagetyp.lower():
        hdulist = fits.open(im.filepath)
        dat = hdulist[0].data
        Bias_list.append(dat)
        hdulist.close

# Then we median combine the images        
Bias_stack = np.stack(Bias_list)
masterbias = np.median(Bias_stack, axis=0)

In [ ]:
# Create the masterdark

# Loop over all the images in our list and check if they are Bias frames, if they are we append them to the Bias_list
Darks_list = []
count = 0
for im in myimages:
    if 'dark frame' in im.imagetyp.lower():
        hdulist = fits.open(im.filepath)
        dat = hdulist[0].data
        Darks_list.append((dat - masterbias)/im.exptime)
        hdulist.close

# Then we median combine the darks
Darks_stack = np.stack(Darks_list)
masterdark = np.median(Darks_stack, axis=0)

In [ ]:
# Let's do the masterflat next

# Loop over all the images in our list and check if they are Bias frames, if they are we append them to the Bias_list
Flats_list = []
count = 0
for im in myimages2:
    if 'flat field' in im.imagetyp.lower():
        hdulist = fits.open(im.filepath)
        dat = hdulist[0].data
        Flats_list.append(dat)
        hdulist.close

# # Lets inspect the Flats
# c = 0
# for obj in Flats_list:
#     fig = figure()
#     frame = fig.add_subplot(1,1,1)
#     norm = ImageNormalize(stretch=SquaredStretch())
#     frame.imshow(obj, interpolation='none', origin='lower', cmap='gray', norm=norm)
#     frame.set_title(f"{c}, min = {np.min(obj)}, max = {np.max(obj)}")
#     show(fig)
#     close(fig)
#     c += 1

# After inspecting the flats we get
Flats_list_sorted = Flats_list[0:4]

In [21]:
# Then we calibrate the flats with masterbias and median combine the images (We don't need to concern with the darks as the dark current can't really build up during flats
Flats_stack = np.stack(Flats_list_sorted-masterbias)
Flats_median = np.median(Flats_stack)
Flats_stack_normalized = Flats_stack/Flats_median
masterflat_to_be_normalized = np.median(Flats_stack_normalized, axis=0)
masterflat = masterflat_to_be_normalized/np.median(masterflat_to_be_normalized)

In [22]:
# Now that we have our master calibration files we can work on our lights, first we calibrate
lights = []
for im in myimages2:
    if 'light frame' in im.imagetyp.lower():
        hdulist = fits.open(im.filepath)
        data = hdulist[0].data.astype(float)
        data -= masterdark*im.exptime + masterbias
        # data /= masterflat   # Don't devide by the masterflat, because it gives weird results due to the dark patches between the emission lines
        lights.append(data)
        hdulist.close

# Let's inspect the lights
# c = 0
# for obj in lights:
#     fig = figure()
#     frame = fig.add_subplot(1,1,1)
#     frame.imshow(obj, interpolation='none', origin='lower', cmap='gray')
#     frame.set_title(f"light {c} with min: {obj.min()}, max: {obj.max()}")
#     show(fig)
#     close(fig)
#     c += 1

In [23]:
lights_sorted_galaxy = lights[32:34]+lights[36:42]
lights_sorted_vega = lights[43:48]
lights_sorted_moon = lights[-5:-1]
lights_calibration = lights[21:25]
lights_sorted_arcturus = lights[29:31]
lights_stack = np.stack(lights_calibration)
masterlight = np.median(lights_stack, axis=0)

lights_stack_moon = np.stack(lights_sorted_moon)
masterlight_moon = np.median(lights_stack_moon, axis=0)

In [24]:
# Now that we have a program which calibrates the images we can start on figuring out where the spectral lines are. This is easiest done by using the flat frames first slice in a vertical line

vert_slice = masterflat[:,1000]

# Based on the plot we want to exclude values with less then 10000 counts
fibre_locations = np.where(vert_slice>3)[0]

# Now we would like to group all of the values in a single fibre into a single median value to reduce noise, although we do clearly see that the values on the outside are significantly less then the ones in the center.
fibre_locations_dict = {}
i = 1
n = "fibre_0"
c = 0

# Exclude outer 2, because poor signal to noise ratio 
# And add weighting, cause these tops be looking hella mid

for obj in fibre_locations:
    try:
        if obj+1 == fibre_locations[i]:
            try:
                fibre_locations_dict[n].append(obj)
            except:
                fibre_locations_dict[n] = []
                fibre_locations_dict[n].append(obj)
        else:
            fibre_locations_dict[n].append(obj)
            c+=1
            n = f'fibre_{c}'
        i+=1

    except:
        fibre_locations_dict[n].append(obj)

In [25]:
# Now that we have the locations of the fibres we make an intensity plot of the median of all the collumns that a fibre consists of.
def find_maximum_slice(horizontal_slice_dict):
    '''
    :purpose: Finds the location of the slice which contains the maximum value
    :input: Dictionary medians of horizontal slices
    :return: The location of the slice which contains the maximum value
    '''
    Total_max = np.max(horizontal_slice_dict['fibre_0'])
    for i in horizontal_slice_dict:
        Max_of_row = np.max(horizontal_slice_dict[i])
        if Max_of_row>Total_max:
            Total_max = Max_of_row
            location = i
    return location

horizontal_slice_dict = {}
horizontal_slice_moon_dict = {}
flat_horizontal_slice_dict = {}
horizontal_slice_list = []

# Loop over all the fibres
for i in fibre_locations_dict:
    # Do the slicing
    horizontal_slice = masterlight[fibre_locations_dict[i],:]
    horizontal_slice_moon = masterlight_moon[fibre_locations_dict[i],:]
    flat_horizontal_slice = masterflat[fibre_locations_dict[i], :]
    # Take the median
    horizontal_slices_median = np.median(horizontal_slice, axis=0)
    horizontal_slices_median_moon = np.median(horizontal_slice_moon, axis=0)
    flat_horizontal_slice_median = np.median(flat_horizontal_slice, axis=0)
    # Add this median line to the dictionary
    horizontal_slice_dict[i] = horizontal_slices_median
    horizontal_slice_moon_dict[i] = horizontal_slices_median_moon
    flat_horizontal_slice_dict[i] = flat_horizontal_slice_median
    # Append list
    horizontal_slice_list.append(horizontal_slices_median)


# We want to save the horizontal_slice_dict and use it in further stuff
location_of_maximum = find_maximum_slice(horizontal_slice_dict)

In [29]:
pixels_xaxis = 2004

# Location of the peaks in our image
a1 = np.array([199,1597,1863])
# Values from literature: Hb = 486.1, Ha = 656.3, O2 = 686.9
c1 = np.array([486.1,656.3,686.9])
a2 = np.arange(0,pixels_xaxis, 1)

def wavelenghtfit(a1,c1):
    ''''
    Function to calculate the wavelenght calibration polynomial
    input: a1 and c1 are arrays where the first is the pixel values found and c1 are the wavelength values
    output: wavelenght calibration polynomial
    '''
    y = np.poly1d(np.polyfit(a1,c1,1))
    return(y)

y = (wavelenghtfit(a1,c1))

wavelenght_calibrated = np.polyval(y, a2)

# Test wavelenght calibration
# fig = figure()
# frame = fig.add_subplot(1,1,1)
# frame.plot(wavelenght_calibrated, flat_calibrated)
# frame.set_title(f"{location_of_maximum} wavelenght calibrated")
# frame.grid()
# show(fig)
# close(fig)